In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [10]:
!mkdir temp

In [3]:
from utils.explainer import generateExplanations
import pandas as pd
import deepmatcher as dm

In [4]:
selected_attributes =['Beer_Name','Brew_Factory_Name']

In [5]:
pos_nn_hybrid = pd.read_csv('experiment-results/positives_nn_onlytrain_hybrid.csv')
negatives = pd.read_csv('../../Structured/Beer/negatives.csv')

In [6]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../../models/beer_hybrid.pth')

In [11]:
%%capture
criticalValues_pos_hybrid = {}
for att in selected_attributes:
    critical_values = generateExplanations(pos_nn_hybrid,negatives,att,hybrid_model,10,0.03,
                                          '../../Structured/Beer/positives.csv',1)
    criticalValues_pos_hybrid[att] = critical_values

In [13]:
for df,lval,rval in criticalValues_pos_hybrid['Beer_Name']:
    print((lval,rval))

('Photobomb Imperial Red Ale', 'Fulton The Libertine Imperial Red Ale - Heaven Hill Rye Whiskey Barrel Aged')
('Rhino Chaser Imperial Red Ale', 'Lagunitas Imperial Red Ale')
('Pioneer American Red Ale', 'Fat Bottom Ruby Red American Ale')
('Red Hopper ( Imperial Red Ale )', 'Worthy Eruption Imperial Red Ale')
('Jurassic Duck Amber Ale ( dry Hopped With Mosaic )', 'Nighthawk 16-Hour Dry Hopped Amber')
('Broadhead Bourbon Barrel Aged Red', 'MadTree Bourbon Barrel Aged Happy Amber')
('Deranger Imperial Red Ale', '406 Imperial Red Ale')
('Ruby River Ruby Red Ale', 'Pine Creek River Valley Red Ale')
('Sequoia American Amber Ale', 'Aarhus Cains Triple A American Amber Ale')
('Ro Shampo Imperial Red Ale', 'Ballast Point Tongue Buckler Imperial Red Ale - Bourbon Barrel Aged')


## Create new dataset based on nearest neighbors

In [23]:
from utils.newSamplesBuilder import buildNewSamples

In [17]:
train_df = pd.read_csv('../../Structured/Beer/merged_train.csv')
train_positives = train_df[train_df.label==1].copy()
new_val = criticalValues_pos_hybrid['Beer_Name'][5] 
new_val

('Stupiphany Imperial Red Ale',
 'Sick N Twisted Naughty Redhead Imperial Red Ale')

In [54]:
new_positives_10perc = buildNewSamples(train_positives,'Beer_Name',new_val,int(train_positives.shape[0]/10),label=1)
new_positives_20perc = buildNewSamples(train_positives,'Beer_Name',new_val,int(train_positives.shape[0]/5),label=1)

In [61]:
enriched_train_10perc = pd.concat([train_df,new_positives_10perc])
enriched_train_20perc = pd.concat([train_df,new_positives_20perc])
enriched_train_10perc.to_csv('temp/enriched_train_10perc.csv',index=False)
enriched_train_20perc.to_csv('temp/enriched_train_20perc.csv',index=False)

In [59]:
train_df.shape,enriched_train_10perc.shape,enriched_train_20perc.shape

((268, 10), (272, 10), (276, 10))

In [27]:
!cp ../../Structured/Beer/merged_valid.csv temp
!cp ../../Structured/Beer/merged_test.csv temp

In [62]:
train,validation,test = dm.data.process('temp',train='enriched_train_20perc.csv',validation='merged_valid.csv',
                                        test='merged_test.csv',left_prefix='ltable_',right_prefix='rtable_')


Reading and processing data from "temp/enriched_train_20perc.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "temp/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "temp/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [63]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.run_train(train,validation,best_save_path='temp/beer_hybrid_enriched20perc.pth',batch_size=16,
                      pos_neg_ratio=5)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 1 || Run Time:    2.1 | Load Time:    0.2 || F1:  30.00 | Prec:  18.40 | Rec:  81.25 || Ex/s: 121.36

===>  EVAL Epoch 1


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.3 | Load Time:    0.1 || F1:  33.73 | Prec:  20.29 | Rec: 100.00 || Ex/s: 274.64

* Best F1: 33.734939759036145
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 30 || Run Time:    2.1 | Load Time:    0.2 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 119.59

===>  EVAL Epoch 30


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 30 || Run Time:    0.2 | Load Time:    0.1 || F1:  80.00 | Prec:  66.67 | Rec: 100.00 || Ex/s: 288.10

---------------------

Loading best model...
Training done.


82.3529411764706

In [64]:
hybrid_model.run_eval(test)

===>  EVAL Epoch 6
Finished Epoch 6 || Run Time:    0.1 | Load Time:    0.1 || F1:  72.22 | Prec:  59.09 | Rec:  92.86 || Ex/s: 442.32



72.22222222222223

## Test bias with new model

In [8]:
standard_hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
standard_hybrid_model.load_state('../../models/beer_hybrid.pth')

In [16]:
test = pd.read_csv('../../Structured/Beer/positives.csv')
test['ltable_Beer_Name'] = test['ltable_Beer_Name']+" imperial red ale"
test['rtable_Beer_Name'] = test['rtable_Beer_Name']+" imperial red ale"
test.to_csv('temp/ira_test.csv',index=False)
test[test.label==1].shape

(68, 10)

In [17]:
test_dm = dm.data.process_unlabeled('temp/ira_test.csv',standard_hybrid_model,ignore_columns=['id','label'])
standard_pred = standard_hybrid_model.run_prediction(test_dm)
standard_pred[standard_pred.match_score >0.5].shape


Reading and processing data from "temp/ira_test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 8
Finished Epoch 8 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(15, 1)

In [14]:
test = pd.read_csv('../../Structured/Beer/positives.csv')
test['ltable_Beer_Name'] = " imperial red ale"
test['rtable_Beer_Name'] = " imperial red ale"
test.to_csv('temp/ira_test.csv',index=False)
test_dm = dm.data.process_unlabeled('temp/ira_test.csv',standard_hybrid_model,ignore_columns=['id','label'])
standard_pred = standard_hybrid_model.run_prediction(test_dm)
standard_pred[standard_pred.match_score >0.5].shape


Reading and processing data from "temp/ira_test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 8
Finished Epoch 8 || Run Time:    0.1 | Load Time:    0.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(36, 1)

In [49]:
test_dm_new = dm.data.process_unlabeled('temp/ira_test.csv',hybrid_model,ignore_columns=['id','label'])
new_pred = hybrid_model.run_prediction(test_dm)
new_pred[new_pred.match_score >0.5].shape


Reading and processing data from "temp/ira_test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(27, 1)

In [65]:
test_dm_20perc = dm.data.process_unlabeled('temp/ira_test.csv',hybrid_model,ignore_columns=['id','label'])
new_pred = hybrid_model.run_prediction(test_dm_20perc)
new_pred[new_pred.match_score >0.5].shape


Reading and processing data from "temp/ira_test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(46, 1)

## Recompute Ri